# Markov Logic Networks to ProbLog

June 2015  
[Wannes Meert](mailto:wannes.meert@cs.kuleuven.be), [Guy Van den Broeck](mailto:guy.vandenbroeck@cs.kuleuven.be), [Anton Dries](mailto:anton.dries@cs.kuleuven.be), DTAI Research Group, KU Leuven

Although the semantics of ProbLog and MLNs are different, you can transform any first-order MLN into a ProbLog program. This is discussed in:

> Daan Fierens, Guy Van den Broeck, Maurice Bruynooghe, and Luc De Raedt.
> "Constraints for probabilistic logic programming."
> In Proceedings of the NIPS probabilistic programming workshop, pp. 1-4. 2012.

The converse is false. For example, a ProbLog program with inductive definitions such as path or reachability cannot be expressed in first-order logic (although you can always reduce to SAT, propositionally).

For an overview of the MLN semantics, we refer to:

> Stanley Kok, Parag Singla, Matthew Richardson and Pedro Domingos (2005). 
> "The Alchemy System for Statistical Relational AI", Technical Report, 
> Department of Computer Science and Engineering, 
> University of Washington, Seattle, WA. 
> http://www.cs.washington.edu/ai/alchemy.


## Using the WFOMC Implementation

The conversion from MLN to ProbLog is built into the Weighted First-Order Model Counting (WFOMC) application available from https://dtai.cs.kuleuven.be/wfomc.

The complete chain looks as follows:

    $ java -jar wfomc.jar --problog-out friendsmoker.pl friendsmoker.mln
    $ echo -e "query(friends(guy,wannes)).\nquery(smokes(guy))." >> friendsmoker.pl
    $ ./problog-cli.py friendsmoker.pl
    	friends(guy,wannes) : 0.45804886
	            smokes(guy) : 0.5

The first step creates a ProbLog theory that is equivalent to the given MLN file. The second stap adds the query statements to the ProbLog file. Finally, ProbLog is called to compute the probabilities of the queries.

## Manual Conversion

It is also possible to perform the computation manually. This is based on the method mentioned in the cProbLog paper and the transformation to weighted first-order model counting explained in:

> Van den Broeck, Guy, Nima Taghipour, Wannes Meert, Jesse Davis, and Luc De Raedt.
> "Lifted probabilistic inference by first-order knowledge compilation."
> In Proceedings of the Twenty-Second international joint conference on Artificial Intelligence-Volume Volume Three, pp. 2178-2185. AAAI Press, 2011.

### MLN to WFOMC

An MLN

    people = {A,B}
    Smokes(people)
    Friends(people,people)
    w Smokes(x) ^ Friends(x,y) => Smokes(y)

is translated to an MLN where only literal formulas have a weight (ignoring the type definitions):

    w R(x,y)
    R(x,y) <=> (Smokes(x) ^ Friends(x,y) => Smokes(y)).


### WFOMC to ProbLog

The WFOMC program above can be written to cProbLog:

    people(a). people(b).
    0.5 :: smokes(X) :- people(X).
    0.5 :: friends(X) :- people(X), people(Y).
    w' :: r(X,Y).
    evidence(r(X,Y) <=> smokes(X) ^ friends(X,Y) => smokes(Y), true) :- people(X), people(Y).

with $w' = \frac{e^w}{e^w+1}$, upercase symbols for variables and lowercase symbols for constants and predicates. The types of the predicates are introduced by adding a `people` predicate that lists the domain elements and linking this to the `smokes` and `friends` predicates. The 0.5 probability is a default for MLNs if a predicate is not introduced explicitely ($\frac{e^0}{e^0+1} = 0.5$). If a sentence `w_s Smokes(x)` is present in the MLN, the 0.5 would be replaced by $w_s' = \frac{e^{w_s}}{e^{w_s}+1}$ for `smokes`.

The current ProbLog implementation does not support first-order logic constraints, so this has to be rewritten to ProbLog rules. This can be achieved by turning the constraint into a CNF and adding a ProbLog clauses `ev :- not_c_i` for every clause `c_i` in the CNF. The `not_c_i` is the negation of clause `c_i`. Finally, you set the evidence on `ev` to false.

    people(a). people(b).
    0.5 :: smokes(X) :- people(X).
    0.5 :: friends(X) :- people(X), people(Y).
    w' :: r(X,Y).
    ev :- people(X), people(Y), \+f_1(X,Y), \+friends(X,Y).
    ev :- people(X), people(Y), \+f_1(X,Y), \+smokes(X).
    ev :- people(Y), smokes(X), \+f_1(Y,X).
    ev :- smokes(X), friends(X,Y), f_1(X,Y), \+smokes(Y).
    evidence(ev,false).


## Partition Function

If you want to compute the partition function of an MLN you can query the probability of the evidence `ev` (do not forget to remove the `evidence(ev,false)` rule. Since the ProbLog probabilities are a normalization of the original weights, you have to denormalize them. This can be done by multiplying the probability of evidence with $1+e^w$ for all grounded sentences that appear in the MLN (or equivalently all groundings of all `r` that appear in ProbLog).

\begin{equation}
    \textit{partitionfunction(program)} = Pr(ev)\cdot \prod_{w = \textit{weight(c)} \land c \in \textit{clauses(grounding)}} (1+e^w)
\end{equation}


## More information

https://dtai.cs.kuleuven.be/problog